In [1]:
# Mount Google Drive to Colab
from google.colab import drive

drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [2]:
!pwd
#!ls -al
%cd '/content/gdrive/My Drive/NLP/Part2'
#!ls -al

/content
/content/gdrive/My Drive/NLP/Part2


# 作業 : 實作英文-德文翻譯機器人
***
## [作業目標]

用 pytorch 實作一個英文-德文翻譯機器人

## [作業目標]

*   語言資料處理
*   使用 LSTM 建構 Encoder: EncoderLSTM
*   使用 LSTM 建構 Decoder: DecoderLSTM
*   搭建 Sequence to Sequence 模型: Seq2Seq
*   撰寫訓練函式
*   撰寫測試函式

## [問題]

在 Colab 實際上執行完這個範例後，請改用 BiLSTM 來建構 Encoder 與 Decoder


## 安裝 spacy

We'll also make use of spaCy to tokenize our data. To install spaCy, follow the instructions here making sure to install both the English and German models with:

In [3]:
!pip uninstall spacy -y
!pip install -U spacy

Uninstalling spacy-2.2.4:
  Successfully uninstalled spacy-2.2.4
     |████████████████████████████████| 12.8MB 251kB/s 
     |████████████████████████████████| 9.1MB 58.6MB/s 
     |████████████████████████████████| 1.1MB 40.2MB/s 
     |████████████████████████████████| 460kB 54.3MB/s 
     |████████████████████████████████| 122kB 60.0MB/s 
  Created wheel for smart-open: filename=smart_open-3.0.0-cp37-none-any.whl size=107098 sha256=c595b6f31a103fcf7fa529211f42a47fc10996c7fe31cfbc484493f7f7f1a395
  Stored in directory: /root/.cache/pip/wheels/18/88/7c/f06dabd5e9cabe02d2269167bcacbbf9b47d0c0ff7d6ebcb78
Successfully built smart-open
  Found existing installation: smart-open 4.2.0
    Uninstalling smart-open-4.2.0:
      Successfully uninstalled smart-open-4.2.0
  Found existing installation: catalogue 1.0.0
    Uninstalling catalogue-1.0.0:
      Successfully uninstalled catalogue-1.0.0
  Found existing installation: srsly 1.0.5
    Uninstalling srsly-1.0.5:
      Successfully uninsta

## 引用需要的模組

In [4]:
import jieba
import torch
import torch.nn as nn
import torch.optim as optim
from torchtext.legacy.datasets import Multi30k
from torchtext.legacy.data import Field, BucketIterator
import numpy as np
import pandas as pd
import spacy
import random
# from torchtext.data.metrics import bleu_score
from pprint import pprint
from torch.utils.tensorboard import SummaryWriter
from torchsummary import summary

## 下載英文預料

In [5]:
!mkdir ./data
!mkdir ./data/multi30k
!python -m spacy download en
!ls ./data/multi30k -al
spacy_english = spacy.load("en_core_web_sm")
!ls ./data/multi30k -al

mkdir: cannot create directory ‘./data’: File exists
mkdir: cannot create directory ‘./data/multi30k’: File exists
2021-04-08 01:23:36.707562: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
⚠ As of spaCy v3.0, shortcuts like 'en' are deprecated. Please use the
full pipeline package name 'en_core_web_sm' instead.
     |████████████████████████████████| 13.7MB 1.3MB/s 
  Found existing installation: en-core-web-sm 2.2.5
    Uninstalling en-core-web-sm-2.2.5:
      Successfully uninstalled en-core-web-sm-2.2.5
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
total 0
total 0


## 下載德語語料

In [6]:
!python -m spacy download de
spacy_de = spacy.load("de_core_news_sm")
!ls ./data/multi30k -al

2021-04-08 01:23:46.902084: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
⚠ As of spaCy v3.0, shortcuts like 'de' are deprecated. Please use the
full pipeline package name 'de_core_news_sm' instead.
     |████████████████████████████████| 19.3MB 8.4MB/s 
✔ Download and installation successful
You can now load the package via spacy.load('de_core_news_sm')
total 0


In [7]:
def tokenize_de(text):
    return [token.text for token in spacy_de.tokenizer(text)]

def tokenize_english(text):
    return [token.text for token in spacy_english.tokenizer(text)]

### Sample Run ###

sample_text = "I love machine learning"
print(tokenize_english(sample_text))

german = Field(tokenize=tokenize_de, lower=True,
               init_token="<sos>", eos_token="<eos>")

english = Field(tokenize=tokenize_english, lower=True,
               init_token="<sos>", eos_token="<eos>")

train_data, valid_data, test_data = Multi30k.splits(exts = (".en", ".en"),
                                                    fields=(german, english))

german.build_vocab(train_data, max_size=10000, min_freq=3)
english.build_vocab(train_data, max_size=10000, min_freq=3)

print(f"Unique tokens in source (german) vocabulary: {len(german.vocab)}")
print(f"Unique tokens in target (en) vocabulary: {len(english.vocab)}")


['I', 'love', 'machine', 'learning']
Unique tokens in source (german) vocabulary: 4587
Unique tokens in target (en) vocabulary: 4556


In [8]:
#word_2_idx = dict(e[2])
#idx_2_word = {}
#for k,v in word_2_idx.items():
#    idx_2_word[v] = k

In [9]:
print(f"Number of training examples: {len(train_data.examples)}")
print(f"Number of validation examples: {len(valid_data.examples)}")
print(f"Number of testing examples: {len(test_data.examples)}")

print(train_data[5].__dict__.keys())
pprint(train_data[5].__dict__.values())

Number of training examples: 29000
Number of validation examples: 1014
Number of testing examples: 1000
dict_keys(['src', 'trg'])
dict_values([['a', 'man', 'in', 'green', 'holds', 'a', 'guitar', 'while', 'the', 'other', 'man', 'observes', 'his', 'shirt', '.'], ['a', 'man', 'in', 'green', 'holds', 'a', 'guitar', 'while', 'the', 'other', 'man', 'observes', 'his', 'shirt', '.']])


In [10]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
BATCH_SIZE = 32

train_iterator, valid_iterator, test_iterator = BucketIterator.splits((train_data, valid_data, test_data), 
                                     batch_size = BATCH_SIZE, 
                                     sort_within_batch=True,
                                     sort_key=lambda x: len(x.src),
                                     device = device)

In [11]:
count = 0
max_len_eng = []
max_len_ger = []
for data in train_data:
    max_len_ger.append(len(data.src))
    max_len_eng.append(len(data.trg))
    if count < 10 :
        print("German - ",*data.src, " Length - ", len(data.src))
        print("English - ",*data.trg, " Length - ", len(data.trg))
        print()
    count += 1

print("Maximum Length of English sentence {} and German sentence {} in the dataset".format(max(max_len_eng),max(max_len_ger)))
print("Minimum Length of English sentence {} and German sentence {} in the dataset".format(min(max_len_eng),min(max_len_ger)))

German -  two young , white males are outside near many bushes .  Length -  11
English -  two young , white males are outside near many bushes .  Length -  11

German -  several men in hard hats are operating a giant pulley system .  Length -  12
English -  several men in hard hats are operating a giant pulley system .  Length -  12

German -  a little girl climbing into a wooden playhouse .  Length -  9
English -  a little girl climbing into a wooden playhouse .  Length -  9

German -  a man in a blue shirt is standing on a ladder cleaning a window .  Length -  15
English -  a man in a blue shirt is standing on a ladder cleaning a window .  Length -  15

German -  two men are at the stove preparing food .  Length -  9
English -  two men are at the stove preparing food .  Length -  9

German -  a man in green holds a guitar while the other man observes his shirt .  Length -  15
English -  a man in green holds a guitar while the other man observes his shirt .  Length -  15

German -  a 

In [12]:
count = 0
for data in train_iterator:
    if count < 1 :
        print("Shapes", data.src.shape, data.trg.shape)
        print()
        print("German - ",*data.src, " Length - ", len(data.src))
        print()
        print("English - ",*data.trg, " Length - ", len(data.trg))
        temp_ger = data.src
        temp_eng = data.trg
        count += 1

Shapes torch.Size([13, 32]) torch.Size([14, 32])

German -  tensor([2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
        2, 2, 2, 2, 2, 2, 2, 2], device='cuda:0') tensor([ 16,   4,   4, 203,   4,   4,   4, 110,   7,  21,   4,   4,   4,   4,
          4,   4,  19,  49,   4, 417,   4,   4,  19,   4,   4,   4,   4,   4,
          4,   4,   4,  16], device='cuda:0') tensor([ 100, 2808,   23,   63,  119,   23,    9,   49,   61,  229,    9,   38,
          15,   63,  682,   15,   72,   22,  865,  190,   15,    9,   17,   15,
          23,    9,    9,    9,   63,   51,   15,   30], device='cuda:0') tensor([1405,    6,    0,   10,  237,   15,   22,   17,   35,   15,    6,   12,
          22, 1805,    9,  148,   53,    4,   10,  371,   10,  501,  603,    6,
          34,    6,   22,   37,    6,   32,    0,   98], device='cuda:0') tensor([   6,   42,   90, 3136,   10,   10,    4,   36,   10,   10,    4,  517,
           4,    0,   31, 1352, 1615,   89,   41,   28,  407

In [13]:
temp_eng_idx = (temp_eng).cpu().detach().numpy()
temp_ger_idx = (temp_ger).cpu().detach().numpy()

In [14]:
df_eng_idx = pd.DataFrame(data = temp_eng_idx, columns = [str("S_")+str(x) for x in np.arange(1, 33)])
df_eng_idx.index.name = 'Time Steps'
df_eng_idx.index = df_eng_idx.index + 1 
# df_eng_idx.to_csv('/content/idx.csv')
df_eng_idx

,S_1,S_2,S_3,S_4,S_5,S_6,S_7,S_8,S_9,S_10,S_11,S_12,S_13,S_14,S_15,S_16,S_17,S_18,S_19,S_20,S_21,S_22,S_23,S_24,S_25,S_26,S_27,S_28,S_29,S_30,S_31,S_32
Time Steps,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2
2,16,4,4,209,4,4,4,110,7,21,4,4,4,4,4,4,19,50,4,417,4,4,19,4,4,4,4,4,4,4,4,16
3,104,2813,24,64,123,24,9,50,61,233,9,38,14,64,683,14,73,22,870,196,14,9,17,14,24,9,9,9,64,53,14,30
4,1409,6,0,10,237,14,22,17,35,14,6,12,22,1809,9,151,54,4,10,375,10,504,606,6,34,6,22,37,6,33,0,100
5,6,43,91,3131,10,10,4,36,10,10,4,521,4,0,32,1359,1624,90,41,28,411,4,199,4,8,4,4,4,29,6,20,6
6,4,12,8,13,45,165,132,129,0,883,197,133,1278,4097,8,6,1969,23,4,7,2978,921,306,1177,2576,132,29,1088,148,4,4,43
7,1409,27,4,964,51,1990,23,15,20,28,10,6,10,0,4,7,15,395,0,326,336,265,717,10,179,23,234,6,11,2872,131,12
8,277,616,365,336,46,12,10,249,121,32,633,43,130,8,149,488,3028,3986,288,330,4,124,6,790,76,37,10,21,22,137,1429,4
9,20,594,694,4,105,7,0,54,6,6,20,12,13,4,37,12,254,6,51,7,38,13,4,6,4,4,8,593,4,6,6,0


In [15]:
idx2word = {idx: word for idx, word in enumerate(english.vocab.itos)}
df_eng_word = pd.DataFrame(columns=[str('S_') + str(x + 1) for x in range(BATCH_SIZE)])
df_eng_word = df_eng_idx.replace(idx2word)
df_eng_word

,S_1,S_2,S_3,S_4,S_5,S_6,S_7,S_8,S_9,S_10,S_11,S_12,S_13,S_14,S_15,S_16,S_17,S_18,S_19,S_20,S_21,S_22,S_23,S_24,S_25,S_26,S_27,S_28,S_29,S_30,S_31,S_32
Time Steps,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,<sos>,<sos>,<sos>,<sos>,<sos>,<sos>,<sos>,<sos>,<sos>,<sos>,<sos>,<sos>,<sos>,<sos>,<sos>,<sos>,<sos>,<sos>,<sos>,<sos>,<sos>,<sos>,<sos>,<sos>,<sos>,<sos>,<sos>,<sos>,<sos>,<sos>,<sos>,<sos>
2,two,a,a,this,a,a,a,four,the,an,a,a,a,a,a,a,people,women,a,2,a,a,people,a,a,a,a,a,a,a,a,two
3,girls,professor,young,person,soccer,young,man,women,brown,elderly,man,group,woman,person,bald,woman,dressed,wearing,hiker,kids,woman,man,are,woman,young,man,man,man,person,little,woman,men
4,coloring,in,<unk>,is,team,woman,wearing,are,dog,woman,in,of,wearing,places,man,carrying,for,a,is,sleeping,is,carries,gathering,in,boy,in,wearing,playing,in,girl,<unk>,stand
5,in,front,sits,parachuting,is,is,a,standing,is,is,a,students,a,<unk>,sitting,buckets,cool,pink,walking,while,throwing,a,under,a,on,a,a,a,blue,in,at,in
6,a,of,on,with,holding,taking,gray,together,<unk>,crying,suit,smiling,headscarf,hamburger,on,in,weather,shirt,a,the,champagne,trash,colored,towel,rollerskates,gray,blue,keyboard,shorts,a,a,front
7,coloring,his,a,skis,up,care,shirt,",",at,while,is,in,is,<unk>,a,the,",",selling,<unk>,adults,onto,bag,lights,is,jumps,shirt,helmet,in,and,sundress,food,of
8,book,class,bridge,onto,one,of,is,posing,something,sitting,speaking,front,working,on,stage,mountains,eagerly,donuts,path,dance,a,along,in,relaxing,over,playing,is,an,wearing,plays,item,a
9,at,giving,overlooking,a,player,the,<unk>,for,in,in,at,of,with,a,playing,of,waiting,in,up,the,group,with,a,in,a,a,on,empty,a,in,in,<unk>


## 用 LSTM 搭建的 Encoder 類別: EncoderLSTM



In [16]:
class EncoderLSTM(nn.Module):
    def __init__(self, input_size, embedding_size, hidden_size, num_layers, p):
        super(EncoderLSTM, self).__init__()

        # Size of the one hot vectors that will be the input to the encoder
        #self.input_size = input_size

        # Output size of the word embedding NN
        #self.embedding_size = embedding_size

        # Dimension of the NN's inside the lstm cell/ (hs,cs)'s dimension.
        self.hidden_size = hidden_size

        # Number of layers in the lstm
        self.num_layers = num_layers

        # Regularization parameter
        self.dropout = nn.Dropout(p)
        self.tag = True

        # Shape --------------------> (5376, 300) [input size, embedding dims]
        self.embedding = nn.Embedding(input_size, embedding_size)

        # Shape -----------> (300, 2, 1024) [embedding dims, hidden size, num layers]
        self.LSTM = nn.LSTM(embedding_size, hidden_size, num_layers, dropout = p)

    # Shape of x (26, 32) [Sequence_length, batch_size]
    def forward(self, x):

        # Shape -----------> (26, 32, 300) [Sequence_length , batch_size , embedding dims]
        embedding = self.dropout(self.embedding(x))

        # Shape --> outputs (26, 32, 1024) [Sequence_length , batch_size , hidden_size]
        # Shape --> (hs, cs) (2, 32, 1024) , (2, 32, 1024) [num_layers, batch_size size, hidden_size]
        outputs, (hidden_state, cell_state) = self.LSTM(embedding)

        return hidden_state, cell_state

input_size_encoder = len(german.vocab)
encoder_embedding_size = 300
hidden_size = 1024
num_layers = 2
encoder_dropout = 0.5

encoder_lstm = EncoderLSTM(input_size_encoder, encoder_embedding_size,
                           hidden_size, num_layers, encoder_dropout).to(device)
print(encoder_lstm)

EncoderLSTM(
  (dropout): Dropout(p=0.5, inplace=False)
  (embedding): Embedding(4587, 300)
  (LSTM): LSTM(300, 1024, num_layers=2, dropout=0.5)
)


## 用 LSTM 搭建的 decoder 類別: DecoderLSTM


In [17]:
class DecoderLSTM(nn.Module):
    def __init__(self, input_size, embedding_size, hidden_size, num_layers, p, output_size):
        super(DecoderLSTM, self).__init__()

        # Size of the one hot vectors that will be the input to the encoder
        #self.input_size = input_size

        # Output size of the word embedding NN
        #self.embedding_size = embedding_size

        # Dimension of the NN's inside the lstm cell/ (hs,cs)'s dimension.
        self.hidden_size = hidden_size

        # Number of layers in the lstm
        self.num_layers = num_layers

        # Size of the one hot vectors that will be the output to the encoder (English Vocab Size)
        self.output_size = output_size

        # Regularization parameter
        self.dropout = nn.Dropout(p)

        # Shape --------------------> (5376, 300) [input size, embedding dims]
        self.embedding = nn.Embedding(input_size, embedding_size)

        # Shape -----------> (300, 2, 1024) [embedding dims, hidden size, num layers]
        self.LSTM = nn.LSTM(embedding_size, hidden_size, num_layers, dropout = p)

        # Shape -----------> (1024, 4556) [embedding dims, hidden size, num layers]
        self.fc = nn.Linear(hidden_size, output_size)

    # Shape of x (32) [batch_size]
    def forward(self, x, hidden_state, cell_state):

        # Shape of x (1, 32) [1, batch_size]
        x = x.unsqueeze(0)

        # Shape -----------> (1, 32, 300) [1, batch_size, embedding dims]
        embedding = self.dropout(self.embedding(x))

        # Shape --> outputs (1, 32, 1024) [1, batch_size , hidden_size]
        # Shape --> (hs, cs) (2, 32, 1024) , (2, 32, 1024) [num_layers, batch_size size, hidden_size] (passing encoder's hs, cs - context vectors)
        outputs, (hidden_state, cell_state) = self.LSTM(embedding, (hidden_state, cell_state))

        # Shape --> predictions (1, 32, 4556) [ 1, batch_size , output_size]
        predictions = self.fc(outputs)

        # Shape --> predictions (32, 4556) [batch_size , output_size]
        predictions = predictions.squeeze(0)

        return predictions, hidden_state, cell_state

input_size_decoder = len(english.vocab)
decoder_embedding_size = 300
hidden_size = 1024
num_layers = 2
decoder_dropout = 0.5
output_size = len(english.vocab)

decoder_lstm = DecoderLSTM(input_size_decoder, decoder_embedding_size,
                           hidden_size, num_layers, decoder_dropout, output_size).to(device)
print(decoder_lstm)

DecoderLSTM(
  (dropout): Dropout(p=0.5, inplace=False)
  (embedding): Embedding(4556, 300)
  (LSTM): LSTM(300, 1024, num_layers=2, dropout=0.5)
  (fc): Linear(in_features=1024, out_features=4556, bias=True)
)


In [18]:
for batch in train_iterator:
    print(batch.src.shape)
    print(batch.trg.shape)
    break

x = batch.trg[1]
print(x)

torch.Size([12, 32])
torch.Size([12, 32])
tensor([   7,    4,  251,    4,    4,    4,    7,   16,    4,    4,   30,   19,
           4,    4, 1274,  113,   16,    4,    4,    4,    4,   16,   16,   16,
         428, 2197,    4,    4,    4,    7,  154,    4], device='cuda:0')


# Sequence to Sequence 類別

In [19]:
class Seq2Seq(nn.Module):
    def __init__(self, Encoder_LSTM, Decoder_LSTM):
        super(Seq2Seq, self).__init__()
        self.Encoder_LSTM = Encoder_LSTM
        self.Decoder_LSTM = Decoder_LSTM

    def forward(self, source, target, tfr=0.5):
        # Shape - Source : (10, 32) [(Sentence length german + some padding), Number of Sentences]
        batch_size = source.shape[1]

        # Shape - Source : (14, 32) [(Sentence length English + some padding), Number of Sentences]
        target_len = target.shape[0]
        target_vocab_size = len(english.vocab)

        # Shape --> outputs (14, 32, 5766) 
        outputs = torch.zeros(target_len, batch_size, target_vocab_size).to(device)

        # Shape --> (hs, cs) (2, 32, 1024) ,(2, 32, 1024) [num_layers, batch_size size, hidden_size] (contains encoder's hs, cs - context vectors)
        hidden_state, cell_state = self.Encoder_LSTM(source)

        # Shape of x (32 elements)
        x = target[0] # Trigger token <SOS>

        for i in range(1, target_len):
            # Shape --> output (32, 5766) 
            output, hidden_state, cell_state = self.Decoder_LSTM(x, hidden_state, cell_state)
            outputs[i] = output
            best_guess = output.argmax(1) # 0th dimension is batch size, 1st dimension is word embedding
            x = target[i] if random.random() < tfr else best_guess # Either pass the next word correctly from the dataset or use the earlier predicted word

        # Shape --> outputs (14, 32, 5766) 
        return outputs


In [20]:
# Hyperparameters

learning_rate = 0.001
writer = SummaryWriter(f"runs/loss_plot")
step = 0

model = Seq2Seq(encoder_lstm, decoder_lstm).to(device)
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

pad_idx = english.vocab.stoi["<pad>"]
criterion = nn.CrossEntropyLoss(ignore_index=pad_idx)

In [21]:
model

Seq2Seq(
  (Encoder_LSTM): EncoderLSTM(
    (dropout): Dropout(p=0.5, inplace=False)
    (embedding): Embedding(4587, 300)
    (LSTM): LSTM(300, 1024, num_layers=2, dropout=0.5)
  )
  (Decoder_LSTM): DecoderLSTM(
    (dropout): Dropout(p=0.5, inplace=False)
    (embedding): Embedding(4556, 300)
    (LSTM): LSTM(300, 1024, num_layers=2, dropout=0.5)
    (fc): Linear(in_features=1024, out_features=4556, bias=True)
  )
)

In [22]:
def translate_sentence(model, sentence, german, english, device, max_length=50):
    spacy_ger = spacy.load('de_core_news_sm')

    if type(sentence) == str:
        tokens = [token.text.lower() for token in spacy_ger(sentence)]
    else:
        tokens = [token.lower() for token in sentence]
    tokens.insert(0, german.init_token)
    tokens.append(german.eos_token)
    text_to_indices = [german.vocab.stoi[token] for token in tokens]
    sentence_tensor = torch.LongTensor(text_to_indices).unsqueeze(1).to(device)

    # Build encoder hidden, cell state
    with torch.no_grad():
        hidden, cell = model.Encoder_LSTM(sentence_tensor)

    outputs = [english.vocab.stoi["<sos>"]]

    for _ in range(max_length):
        previous_word = torch.LongTensor([outputs[-1]]).to(device)

        with torch.no_grad():
            output, hidden, cell = model.Decoder_LSTM(previous_word, hidden, cell)
            best_guess = output.argmax(1).item()

        outputs.append(best_guess)

        # Model predicts it's the end of the sentence
        if output.argmax(1).item() == english.vocab.stoi["<eos>"]:
            break

    translated_sentence = [english.vocab.itos[idx] for idx in outputs]
    return translated_sentence[1:]

# 用來評估模型的函式: bleu
def bleu(data, model, german, english, device):
    targets = []
    outputs = []

    for example in data:
        src = vars(example)["src"]
        trg = vars(example)["trg"]

        prediction = translate_sentence(model, src, german, english, device)
        prediction = prediction[:-1]  # remove <eos> token

        targets.append([trg])
        outputs.append(prediction)

    return bleu_score(outputs, targets)

def checkpoint_and_save(model, best_loss, epoch, optimizer, epoch_loss):
    print('saving')
    print()
    state = {'model': model,'best_loss': best_loss,'epoch': epoch,'rng_state': torch.get_rng_state(), 'optimizer': optimizer.state_dict(),}
    torch.save(state, './checkpoint-NMT')
    torch.save(model.state_dict(),'./checkpoint-NMT-SD')

In [23]:
epoch_loss = 0.0
num_epochs = 100
best_loss = 999999
best_epoch = -1
sentence1 = "ein mann in einem blauen hemd steht auf einer leiter und putzt ein fenster"
ts1  = []

for epoch in range(num_epochs):
    print("Epoch - {} / {}".format(epoch+1, num_epochs))
    model.eval()
    translated_sentence1 = translate_sentence(model, sentence1, german, english, device, max_length=50)
    print(f"Translated example sentence 1: \n {translated_sentence1}")
    ts1.append(translated_sentence1)

    model.train(True)
    for batch_idx, batch in enumerate(train_iterator):
        input = batch.src.to(device)
        target = batch.trg.to(device)

        # Pass the input and target for model's forward method
        output = model(input, target)
        output = output[1:].reshape(-1, output.shape[2])
        target = target[1:].reshape(-1)

        # Clear the accumulating gradients
        optimizer.zero_grad()

        # Calculate the loss value for every epoch
        loss = criterion(output, target)

        # Calculate the gradients for weights & biases using back-propagation
        loss.backward()

        # Clip the gradient value is it exceeds > 1
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1)

        # Update the weights values using the gradients we calculated using bp 
        optimizer.step()
        step += 1
        epoch_loss += loss.item()
        writer.add_scalar("Training loss", loss, global_step=step)

        if epoch_loss < best_loss:
            best_loss = epoch_loss
            best_epoch = epoch
            checkpoint_and_save(model, best_loss, epoch, optimizer, epoch_loss) 
            if ((epoch - best_epoch) >= 10):
                print("no improvement in 10 epochs, break")
                break
        print("Epoch_Loss - {}".format(loss.item()))
        print()

    print(epoch_loss / len(train_iterator))

    # score = bleu(test_data[1:100], model, german, english, device)
    # print(f"Bleu score {score*100:.2f}")

串流輸出內容已截斷至最後 5000 行。

Epoch_Loss - 0.06253789365291595

Epoch_Loss - 0.030938968062400818

Epoch_Loss - 0.1454392373561859

Epoch_Loss - 0.02218719571828842

Epoch_Loss - 0.03533252328634262

Epoch_Loss - 0.041137225925922394

Epoch_Loss - 0.035738714039325714

Epoch_Loss - 0.058834824711084366

Epoch_Loss - 0.019632626324892044

Epoch_Loss - 0.054932884871959686

Epoch_Loss - 0.08425801247358322

Epoch_Loss - 0.03207653388381004

Epoch_Loss - 0.06102989241480827

Epoch_Loss - 0.06541157513856888

Epoch_Loss - 0.1432083249092102

Epoch_Loss - 0.040203992277383804

Epoch_Loss - 0.0752515122294426

Epoch_Loss - 0.1390007734298706

Epoch_Loss - 0.014605768956243992

Epoch_Loss - 0.14492714405059814

Epoch_Loss - 0.02799791470170021

Epoch_Loss - 0.13483192026615143

Epoch_Loss - 0.014059946872293949

Epoch_Loss - 0.04145083203911781

Epoch_Loss - 0.0659397542476654

Epoch_Loss - 0.16382552683353424

Epoch_Loss - 0.06465761363506317

Epoch_Loss - 0.09820222109556198

Epoch_Loss - 0.0480785